# 🎲 Generación de Datos Sintéticos con LLMs

Objetivo: usar IA generativa para crear datos realistas de prueba, aumentar datasets, anonimizar datos sensibles, y generar casos edge para testing.

- Duración: 90 min
- Dificultad: Media
- Stack: OpenAI, Faker, SDV

## 1. Generación simple de registros

In [ ]:
import os
import json
from openai import OpenAI

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def generate_synthetic_records(schema: dict, count: int) -> list:
    """Genera registros sintéticos según esquema."""
    prompt = f'''
Genera {count} registros de datos realistas en formato JSON según este esquema:

{json.dumps(schema, indent=2)}

Devuelve un array JSON con {count} objetos.
'''
    
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.8
    )
    
    return json.loads(resp.choices[0].message.content)

# Esquema de clientes
customer_schema = {
    'customer_id': 'int',
    'name': 'string (nombre completo realista)',
    'email': 'string (email válido)',
    'age': 'int (18-75)',
    'city': 'string (ciudad de USA)',
    'signup_date': 'date (YYYY-MM-DD, últimos 2 años)'
}

customers = generate_synthetic_records(customer_schema, 5)
print(json.dumps(customers, indent=2))

## 2. Generación con contexto de negocio

In [ ]:
def generate_sales_data(num_records: int, context: str) -> list:
    """Genera datos de ventas con contexto."""
    prompt = f'''
Contexto de negocio: {context}

Genera {num_records} transacciones de venta realistas con:
- transaction_id (UUID)
- date (últimos 30 días)
- product (nombre coherente con el contexto)
- quantity (int)
- unit_price (float)
- total (quantity * unit_price)
- payment_method (cash, card, online)

JSON array:
'''
    
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.7
    )
    
    return json.loads(resp.choices[0].message.content)

sales = generate_sales_data(
    num_records=3,
    context='Tienda de electrónica especializada en laptops y accesorios'
)

for sale in sales:
    print(sale)

## 3. Generación de casos edge

In [ ]:
def generate_edge_cases(normal_schema: dict) -> list:
    """Genera casos extremos para testing."""
    prompt = f'''
Genera 10 casos edge/extremos para testing basados en este esquema:

{json.dumps(normal_schema, indent=2)}

Incluye casos como:
- Valores nulos
- Strings vacíos
- Números negativos/cero
- Fechas futuras/pasadas extremas
- Caracteres especiales
- Valores muy largos

JSON array con campo adicional 'edge_case_type':
'''
    
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.5
    )
    
    return json.loads(resp.choices[0].message.content)

edge_cases = generate_edge_cases(customer_schema)
print(f'Generados {len(edge_cases)} casos edge:\n')
for case in edge_cases[:3]:
    print(f"Tipo: {case.get('edge_case_type')}")
    print(case)
    print()

## 4. Anonimización inteligente

In [ ]:
def anonymize_data(sensitive_records: list) -> list:
    """Anonimiza datos manteniendo realismo."""
    prompt = f'''
Anonimiza estos registros manteniendo la estructura y realismo:
- Reemplaza nombres con nombres ficticios
- Cambia emails pero mantén el formato
- Altera IDs pero mantén el tipo
- Preserva patrones estadísticos (edades, fechas)

Datos originales:
{json.dumps(sensitive_records, indent=2)}

Datos anonimizados (mismo formato JSON):
'''
    
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.7
    )
    
    return json.loads(resp.choices[0].message.content)

real_data = [
    {'id': 12345, 'name': 'John Smith', 'email': 'john@company.com', 'salary': 75000},
    {'id': 12346, 'name': 'Maria Garcia', 'email': 'maria@company.com', 'salary': 82000}
]

anonymized = anonymize_data(real_data)
print('Original:', real_data[0])
print('Anonimizado:', anonymized[0])

## 5. Aumento de datos (data augmentation)

In [ ]:
def augment_dataset(original_records: list, target_count: int) -> list:
    """Aumenta dataset generando variaciones."""
    prompt = f'''
Tienes estos {len(original_records)} registros originales:

{json.dumps(original_records, indent=2)}

Genera {target_count} registros nuevos similares pero con variaciones realistas.
Mantén distribuciones y patrones.

JSON array:
'''
    
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.8
    )
    
    return json.loads(resp.choices[0].message.content)

original_products = [
    {'name': 'Laptop Pro 15', 'category': 'Electronics', 'price': 1299},
    {'name': 'Wireless Mouse', 'category': 'Accessories', 'price': 29}
]

augmented = augment_dataset(original_products, 5)
print(f'Dataset aumentado a {len(augmented)} registros:\n')
for item in augmented:
    print(item)

## 6. Generación de series temporales

In [ ]:
def generate_timeseries(metric_name: str, days: int, pattern: str) -> list:
    """Genera serie temporal sintética."""
    prompt = f'''
Genera una serie temporal de {days} días para la métrica: {metric_name}
Patrón: {pattern}

Formato JSON:
[
  {{"date": "YYYY-MM-DD", "value": float}},
  ...
]

Los valores deben seguir el patrón descrito.
'''
    
    resp = client.chat.completions.create(
        model='gpt-4',
        messages=[{'role':'user','content':prompt}],
        temperature=0.5
    )
    
    return json.loads(resp.choices[0].message.content)

revenue_data = generate_timeseries(
    metric_name='daily_revenue',
    days=7,
    pattern='Tendencia creciente con picos los fines de semana, valores entre 5000-15000'
)

for entry in revenue_data:
    print(f"{entry['date']}: ${entry['value']:,.2f}")

## 7. Integración con Faker

In [ ]:
# pip install faker
from faker import Faker
import pandas as pd

fake = Faker()

def generate_hybrid_dataset(count: int) -> pd.DataFrame:
    """Combina Faker (estructura) + LLM (semántica)."""
    # Estructura base con Faker
    base_data = [{
        'user_id': fake.uuid4(),
        'name': fake.name(),
        'email': fake.email(),
        'city': fake.city()
    } for _ in range(count)]
    
    # Enriquecer con LLM (ej: generar bio coherente)
    for record in base_data:
        prompt = f"Genera una bio de 1 frase para {record['name']}, vive en {record['city']}, trabaja en tech."
        resp = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=[{'role':'user','content':prompt}],
            temperature=0.7,
            max_tokens=50
        )
        record['bio'] = resp.choices[0].message.content.strip()
    
    return pd.DataFrame(base_data)

df_hybrid = generate_hybrid_dataset(3)
print(df_hybrid)

## 8. Buenas prácticas

- **Validación**: verifica que datos sintéticos cumplan constraints.
- **Distribuciones**: compara estadísticas con datos reales.
- **Diversidad**: usa temperature alta para mayor variedad.
- **Lotes**: genera en batches para eficiencia.
- **Costos**: usa GPT-3.5 para volumen, GPT-4 para calidad.
- **Complementar**: combina LLMs con bibliotecas tradicionales (Faker, SDV).
- **Testing**: valida con Great Expectations.

## 9. Ejercicios

1. Genera un dataset completo de e-commerce (clientes, productos, pedidos) con relaciones coherentes.
2. Crea un generador de logs de aplicación realistas para testing de pipelines.
3. Implementa anonimización que preserve privacidad diferencial.
4. Construye un augmentador de datos para entrenar modelos de ML con pocos ejemplos.